In [1]:
%%capture --no-stderr
%pip install --quiet -U langchain_openai langchain_core langgraph langchain_community langchain_anthropic langchain_experimental faiss-cpu tiktoken pypdf langchain openai langchain_anthropic langchain_experimental


[notice] A new release of pip is available: 23.2.1 -> 25.1.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [4]:
# Importación de librerias
from langchain.agents.agent import AgentExecutor

# LangChain
from langchain_openai import ChatOpenAI, OpenAIEmbeddings
from langchain.tools.retriever import create_retriever_tool
from langchain.vectorstores import FAISS
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain_community.document_loaders import PyPDFLoader
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.messages import AIMessage

# Utilidades
from typing import TypedDict, Optional, Dict, List, Literal, Annotated
from dotenv import load_dotenv, find_dotenv
import tiktoken
from IPython.display import Image, display

In [5]:
# Carga de variables de entorno
dotenv_path = find_dotenv()
load_dotenv(dotenv_path)

True

In [ ]:
def count_tokens_in_docs(docs, model_name="gpt-4o"):
    """
    Calcula la cantidad total de tokens para una lista de documentos
    usando la librería tiktoken.

    :param docs: Lista de documentos (por ejemplo, la salida de un loader PDF).
    :param model_name: Nombre del modelo para el que queremos simular el conteo.
    :return: Número total de tokens en el contenido de los documentos.
    """
    # Obtenemos la codificación que usa el modelo seleccionado
    encoding = tiktoken.encoding_for_model(model_name)

    total_tokens = 0
    for doc in docs:
        # doc.page_content es el texto de cada página/documento
        text = doc.page_content
        total_tokens += len(encoding.encode(text))

    return total_tokens

In [ ]:
# 2. CARGA DEL PDF
# ----------------------------------------------------------------------------
file_path = "guia_condominio_altavista_completa.pdf"
loader = PyPDFLoader(file_path)
docs = loader.load()

In [ ]:
# Cantidad de documentos extraídos (páginas, secciones, etc. según PyPDFLoader)
print("Número de docs cargados:", len(docs))

# Opcional: vista previa de los primeros 100 caracteres de la primera página
print("Preview contenido:", docs[0].page_content[0:100])
print("Metadata:", docs[0].metadata)

In [ ]:
# 3. CONTAR TOKENS ANTES DE HACER NADA
# ----------------------------------------------------------------------------
tokens_en_documento = count_tokens_in_docs(docs, model_name="gpt-4o")
print("Tokens en el documento completo:", tokens_en_documento)

In [ ]:
text_splitter = RecursiveCharacterTextSplitter(chunk_size=500, chunk_overlap=100)
splits = text_splitter.split_documents(docs)

In [ ]:
# Crea los embeddings
embeddings = OpenAIEmbeddings()
vectorstore = FAISS.from_documents(splits, embeddings)

# Crea el retriever
# retriever = vectorstore.as_retriever(search_kwargs={"k": 3})
retriever = vectorstore.as_retriever(search_kwargs={"k": 10, "score_threshold": 0.3})

In [ ]:
# Modelo utilizado
llm = ChatOpenAI(model="gpt-4o")


In [11]:
from langchain_core.messages import HumanMessage
from langchain_openai import ChatOpenAI, OpenAIEmbeddings
from langchain.vectorstores import FAISS
from langchain.tools.retriever import create_retriever_tool
from langchain.agents import create_tool_calling_agent, AgentExecutor
from langchain_core.prompts import ChatPromptTemplate
from langchain_community.document_loaders import PyPDFLoader
from langchain_text_splitters import RecursiveCharacterTextSplitter

# --- Cargar el PDF y construir el retriever ---
pdf_path = "guia_condominio_altavista_completa.pdf"
loader = PyPDFLoader(pdf_path)
docs = loader.load()

text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=100)
splits = text_splitter.split_documents(docs)

embeddings = OpenAIEmbeddings()
vectorstore = FAISS.from_documents(splits, embeddings)
retriever = vectorstore.as_retriever(search_kwargs={"k": 4})

# --- Crear el tool con el retriever ---
retriever_tool = create_retriever_tool(
    retriever,
    name="info_departamentos",
    description="Usa este tool para responder preguntas sobre el condominio Altavista, incluyendo departamentos, modelos, precios, orientación, bodegas, estacionamientos y fechas de entrega."
)

tools = [retriever_tool]

# --- Crear el agente ---
llm = ChatOpenAI(model="gpt-4o", temperature=0)
prompt = ChatPromptTemplate.from_messages([
    ("system",
     "Eres un asistente virtual que responde exclusivamente en base al documento del condominio Altavista. Usa herramientas si es necesario para encontrar información relevante."),
    ("human", "{input}"),
    ("placeholder", "{agent_scratchpad}")
])

agent = create_tool_calling_agent(llm, tools, prompt)
executor = AgentExecutor(agent=agent, tools=tools, verbose=True)

# --- Probar ---
if __name__ == "__main__":
    pregunta = input("👤 Usuario: ")
    respuesta = executor.invoke({"input": pregunta})
    print("🤖 Agente:", respuesta["output"])




> Entering new AgentExecutor chain...

Invoking: `info_departamentos` with `{'query': 'ubicación del proyecto Altavista'}`


Guía Informativa - Condominio Altavista Norte
1. Introducción
Bienvenido a la guía informativa del proyecto Altavista Norte, un moderno condominio diseñado para ofrecer
comodidad, conectividad y una excelente calidad de vida. Este documento entrega información esencial
sobre el condominio, los departamentos disponibles, valores referenciales, formas de financiamiento,
características de construcción y respuestas a preguntas frecuentes de nuestros clientes potenciales. Está
pensado para responder a todas las inquietudes que pueden surgir en tu proceso de compra.
2. Información del Condominio
Nombre del proyecto: Altavista Norte
Dirección: Av. Los Valles 1432, Maipú, Región Metropolitana, Santiago.
Cantidad de torres: 4 torres de 15 pisos cada una.
Cantidad de departamentos por torre: 10 por piso (600 en total).
Orientaciones disponibles: norte, oriente, poniente